### Importing libraries

In [60]:
%reset

In [1]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import json
import difflib

### Preprocessing

In [2]:
dataset=pd.read_csv("tmdb_5000_movies/tmdb_5000_movies.csv")
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             4803 non-null   int64  
 1   genres             4803 non-null   object 
 2   keywords           4803 non-null   object 
 3   original_language  4803 non-null   object 
 4   original_title     4803 non-null   object 
 5   overview           4800 non-null   object 
 6   popularity         4803 non-null   float64
 7   status             4803 non-null   object 
 8   title              4803 non-null   object 
 9   vote_average       4803 non-null   float64
dtypes: float64(2), int64(1), object(7)
memory usage: 375.4+ KB


In [3]:
dataset["genres"][2]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 80, "name": "Crime"}]'

In [63]:
dataset.head(5)

,budget,genres,keywords,original_language,original_title,overview,popularity,status,title,vote_average
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,Released,Avatar,7.2
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,Released,Pirates of the Caribbean: At World's End,6.9
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,Released,Spectre,6.3
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,Released,The Dark Knight Rises,7.6
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,Released,John Carter,6.1


In [4]:
def genre_ectractor(data):
    data=data[1:][:-1]
    temp=defaultdict(list)
    i=0
    for word in data:
        if word!="}":
            temp[i].append(word)
        elif word=="}":
            temp[i].append("}")
            i+=1
        elif word==",":
            continue
    result=[]
    for key in temp.keys():
        # print(temp[key])
        convert="".join(temp[key])
        if convert[0]==",":
            convert=convert[1:]
        result.append(json.loads(convert)["name"])
    for i in range(len(result)):
        if result[i]=="Science Fiction": #just eliminating some gaps
            result[i]="Science-Fiction"
    return " ".join(result)
# print(genre_ectractor(dataset["genres"][10]))
dataset.genres=dataset.genres.apply(lambda data:genre_ectractor(data))

In [94]:
dataset.head(2)

,budget,genres,keywords,original_language,original_title,overview,popularity,status,title,vote_average
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,Released,Avatar,7.2
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,Released,Pirates of the Caribbean: At World's End,6.9


In [95]:
dataset["budget"][0] #237 million

237000000

In [67]:
# from sklearn.preprocessing import StandardScaler
# budget_scale=StandardScaler()
# budget_scale.fit(pd.array(dataset["budget"]).reshape(-1,1))
# dataset["budget"]=budget_scale.transform(pd.array(dataset["budget"]).reshape(-1,1))
# ingnoring buget and popularity feature beacuse it may cause very sparse data if we applied tfidf

In [5]:
dataset.drop(["budget","popularity","title","vote_average"],axis=1,inplace=True)

In [97]:
dataset.head(3)

,genres,keywords,original_language,original_title,overview,status
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",Released
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",Released
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,Released


In [6]:
def keyword_ectractor(data):
    data=data[1:][:-1]
    temp=defaultdict(list)
    i=0
    for word in data:
        if word!="}":
            temp[i].append(word)
        elif word=="}":
            temp[i].append("}")
            i+=1
        elif word==",":
            continue
    result=[]
    for key in temp.keys():
        # print(temp[key])
        convert="".join(temp[key])
        if convert[0]==",":
            convert=convert[1:]
        result.append(json.loads(convert)["name"])
    final_res=[]
    for keyword in result:
        splitted=keyword.split(" ")
        if len(splitted)==1:
            final_res.append(keyword)
        else:
            final_res.append("-".join(splitted))
    return " ".join(final_res)


In [7]:
dataset["keywords"]=dataset["keywords"].apply(lambda data:keyword_ectractor(data))

In [100]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   genres             4803 non-null   object
 1   keywords           4803 non-null   object
 2   original_language  4803 non-null   object
 3   original_title     4803 non-null   object
 4   overview           4800 non-null   object
 5   status             4803 non-null   object
dtypes: object(6)
memory usage: 225.3+ KB


In [8]:
#some missing values in overview are found
dataset["overview"]=dataset["overview"].fillna("")

In [8]:
dataset

,genres,keywords,original_language,original_title,overview,status
0,Action Adventure Fantasy Science-Fiction,culture-clash future space-war space-colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",Released
1,Adventure Fantasy Action,ocean drug-abuse exotic-island east-india-trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",Released
2,Action Adventure Crime,spy based-on-novel secret-agent sequel mi6 bri...,en,Spectre,A cryptic message from Bond’s past sends him o...,Released
3,Action Crime Drama Thriller,dc-comics crime-fighter terrorist secret-ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,Released
4,Action Adventure Science-Fiction,based-on-novel mars medallion space-travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",Released
...,...,...,...,...,...,...
4798,Action Crime Thriller,united-states–mexico-barrier legs arms paper-k...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,Released
4799,Comedy Romance,,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,Released
4800,Comedy Drama Romance TV Movie,date love-at-first-sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",Released
4801,,,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,Released


In [9]:
#combining all data
combined_features=dataset.genres+" "+dataset.keywords+" "+dataset.original_language+" "+dataset.original_title+" "+dataset.overview+" "+dataset.status

In [10]:
combined_features

0       Action Adventure Fantasy Science-Fiction cultu...
1       Adventure Fantasy Action ocean drug-abuse exot...
2       Action Adventure Crime spy based-on-novel secr...
3       Action Crime Drama Thriller dc-comics crime-fi...
4       Action Adventure Science-Fiction based-on-nove...
                              ...                        
4798    Action Crime Thriller united-states–mexico-bar...
4799    Comedy Romance  en Newlyweds A newlywed couple...
4800    Comedy Drama Romance TV Movie date love-at-fir...
4801      en Shanghai Calling When ambitious New York ...
4802    Documentary obsession camcorder crush dream-gi...
Length: 4803, dtype: object

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
tfidf.fit(combined_features)

TfidfVectorizer()

In [11]:
new_df = tfidf.transform(combined_features)

In [105]:
new_df

<4803x24403 sparse matrix of type '<class 'numpy.float64'>'
	with 267377 stored elements in Compressed Sparse Row format>

### K-means Model

In [79]:
# interia=[] #sum of the squred distance between the datapoints and the centroid 
# for cluster in range(1,50):
#     model=KMeans(n_clusters=cluster,n_init=5,init='k-means++',max_iter=100).fit(new_df)
#     #n_init makes the model to choose best centriods by randomly selecting 5 times
#     #‘k-means++’ : selects initial cluster centers for k-mean clustering in a smart way to speed up convergence. See section Notes in k_init for more details.(from sklearn documentation)
#     interia.append(model.inertia_) 

# #for using this downgrade the numpy version to 1.21.5 or 1.21.4 or else we may experiene error(Thanks to stackoverflow)


In [80]:
# plt.plot(range(1, 50), interia)  
# plt.title('The Elbow Method Graph')  
# plt.xlabel('Number of clusters(k)')  
# plt.ylabel('inertia')  
# plt.show()  

No elbow curve found <br>
Coclusion  :
Its not a suitable model for this problem so i am moving to cosine similarity

#### Cosine-similarity method for content based clustering

In [12]:
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
similarity_table=cosine_similarity(new_df) #returns array of cosine similarity between movies #new_df is tfidf transformed   

In [13]:
similarity_table.shape

(4803, 4803)

In [14]:
print(dataset.loc[dataset.original_title=="Pirates of the Caribbean: At World's End"].index[0])

1


In [15]:
new_df=pd.DataFrame.sparse.from_spmatrix(new_df)
#4803x24403


In [16]:
new_df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,24393,24394,24395,24396,24397,24398,24399,24400,24401,24402
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
def similarity_predictor(data):
    closly_matched=difflib.get_close_matches(data,dataset.original_title.to_list()) #this function has been used to match input movie name with all similar movie names we can do that with a for but handling case sensitivity and spacing mistakes make our task tuff that why we use this library 
    print(closly_matched)
    confirmation=int(input("enter the index value for movie"))
    exact_match=closly_matched[confirmation]
    ##finding the vector for this movie from our dataset
    index_num = dataset.loc[dataset.original_title==exact_match].index[0]
    print("vector of our movie",new_df.iloc[index_num])
    similarity_scores=list(enumerate(similarity_table[index_num])) # finding the index value in similarity table

    res = sorted(similarity_scores, key = lambda x: x[1], reverse = True)[:5]
    print(res)
    print("Predictions are")
    for i in res:
        print(dataset.iloc[i[0]]["original_title"])
    
movie_name="Iron man"
similarity_predictor(movie_name)

['Iron Man', 'Iron Man 3', 'Iron Man 2']
vector of our movie 0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
24398    0.0
24399    0.0
24400    0.0
24401    0.0
24402    0.0
Name: 68, Length: 24403, dtype: Sparse[float64, 0]
[(68, 1.0), (31, 0.5344950480803894), (79, 0.4779866714764942), (7, 0.2912001462137835), (26, 0.26199224987398356)]
Predictions are
Iron Man
Iron Man 3
Iron Man 2
Avengers: Age of Ultron
Captain America: Civil War


# actually we are finding the cosine similarity between the feature vectors of the movies